In [1]:
import pandas as pd
import pdfplumber as pr
import re
import os
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [2]:
PDF_files = []
input_path = r'../input'
for x in os.listdir(input_path):
    if '.pdf' and '年2月' in x:
        PDF_files.append(x)
#這個檔案不要刪除，很重要
GFI = pd.read_excel(r'../input\202201 政府基金運用資訊 v2.0.xlsx',sheet_name = 'Onshore (Raw data)',skiprows=1)

In [3]:
#勞退新制
LPF_LIST_ORI = GFI[GFI['Fund']=='LPF']['Mandate Type'].unique()
#勞退舊制
LRF_LIST_ORI = GFI[GFI['Fund']=='LRF']['Mandate Type'].unique()
#勞保
LIF_LIST = GFI[GFI['Fund']=='LIF']['Mandate Type'].unique()
#國保
NPIF_LIST = GFI[GFI['Fund']=='NPIF']['Mandate Type'].unique()
#退撫
PSPF_LIST = GFI[GFI['Fund']=='PSPF']['Mandate Type'].unique()

In [52]:
勞退_PDF = pr.open(input_path+'/勞工退休基金-111年2月.pdf')
#從0開始
#是一個list
ps              = 勞退_PDF.pages
#用range配len跑for迴圈
pg = ps[13]
page_text = pg.extract_text()
print(page_text[:])
print('國內 106 年第一次委託經營' in page_text)

                      國內 106 年第一次委託經營            
單位：新臺幣元
委任迄今 
111年截至2月 
(107.2.7~111.2.28) 
投信名稱  委託金額  目前淨資產  累積投資報酬率
投資報酬 委託經 目標報酬率
(％) 
率(％)  營排名  (％) 
台新  4,500,000,000   6,781,884,223   -4.46   44.48  4 
 
安聯  7,500,000,000   -3.88   86.88  1 
12,213,860,281  
保德信  5,000,000,000   6,776,107,545   -3.48   34.21  6 
24.78 
國泰  4,500,000,000   6,194,270,840    0.57   36.65  5 
群益  4,750,000,000   6,903,701,354   -5.11   46.18  3 
匯豐中華  5,000,000,000   7,327,222,816   -0.42   48.11  2 
合計  31,250,000,000 4 6,197,047,059   -2.99   49.63  － 
註：本批次為絕對報酬型，契約目標報酬率係以臺灣證交所公告之最近5年年底股票集中市場平均殖
利率加計200個基本點為基準。 
(二) 111 年 2 月被提前收回之國內委外投資帳戶 
 
    1.基金國內委外投資被提前收回帳戶經理人名單及收回原因 
     新制勞工退休基金：無 
                                             
     舊制勞工退休基金：無 
2.受託機構經管本基金帳戶被主管機關懲處之相關資訊 
 
     新制勞工退休基金：無 
 
   
       舊制勞工退休基金：無 
 
 
 
五、基金國外委外投資概況表 
   (一)基金國外委外投資績效、績效指標及排名表 
  1.新制勞工退休基金      
  14 
True


In [11]:
LPF_LIST_ORI

array(['96年第一次委託經營（續約4）', '97年第一次委託經營（續約4）', '98 年第一次委託經營（續約4）',
       '99 年第二次委託經營（續約2） ', '100 年第一次委託經營( 續約2 )', '101 年第一次委託經營( 續約 2)',
       '101 年第二次委託經營( 續約 2)', '102 年第二次委託經營 ( 續約 )',
       '103 年第一次委託經營第二期(續約)', '103 年第二次委託經營', '104 年第一次委託經營第一期( 續約 )',
       '106 年第一次委託經營', '107 年第一次委託經營', '107 年第二次委託經營', '109 年第一次委託經營',
       '110年第一次委託經營'], dtype=object)

In [60]:
LPF_LIST = []
LRF_LIST = []
有新制勞退目標基金的頁數 = []
新制勞退目標基金的名稱   = []
NEW_FUND_NAMES         = []

有舊制勞退目標基金的頁數 = []
舊制勞退目標基金的名稱   = []
OLD_FUND_NAMES         = []

for PDF_file in PDF_files:
    if '勞工退休基金' in PDF_file:
        勞退_PDF = pr.open(input_path+'/'+PDF_file)
        #從0開始
        #是一個list
        ps              = 勞退_PDF.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[p]
            page_text = pg.extract_text()
            if p < 11:
                #新制，將PDF檔案中的基金名稱改成excel內的
                FUND_NAME = 'LPF'
                if '96 年第一次委託經營(續約 4)' in page_text:
                    print('96 年第一次委託經營(續約 4) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('96 年第一次委託經營(續約 4)','96年第一次委託經營（續約4）')
                    LPF_LIST.append('96年第一次委託經營（續約4）')
                if '97 年第一次委託經營(續約 4)' in page_text:
                    print('97 年第一次委託經營(續約 4) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('97 年第一次委託經營(續約 4)','97年第一次委託經營（續約4）')
                    LPF_LIST.append('97年第一次委託經營（續約4）')
                if '98 年第一次委託經營(續約 4)' in page_text:
                    print('98 年第一次委託經營(續約 4) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('98 年第一次委託經營(續約 4)','98 年第一次委託經營（續約4）')
                    LPF_LIST.append('98 年第一次委託經營（續約4）')
                if '99 年第二次委託經營(續約 2)' in page_text:
                    print('99 年第二次委託經營(續約 2) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('99 年第二次委託經營(續約 2)','99 年第二次委託經營（續約2） ')
                    LPF_LIST.append('99 年第二次委託經營（續約2） ')
                if '100 年第一次委託經營(續約 2)(相對報酬)' in page_text:
                    print('100 年第一次委託經營(續約 2)(相對報酬) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('100 年第一次委託經營(續約 2)(相對報酬)','100 年第一次委託經營( 續約2 )')
                    LPF_LIST.append('98-1 全球被動股票型（ 續約2）')
                if '101 年第一次委託經營(續約 2)(相對報酬)' in page_text:
                    print('101 年第一次委託經營(續約 2)(相對報酬) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('101 年第一次委託經營(續約 2)(相對報酬)','101 年第一次委託經營( 續約 2)')
                    LPF_LIST.append('101 年第一次委託經營( 續約 2)')
                if '101 年第二次委託經營(續約 2)' in page_text:
                    print('101 年第二次委託經營(續約 2) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('101 年第二次委託經營(續約 2)','101 年第二次委託經營( 續約 2)')
                    LPF_LIST.append('101 年第二次委託經營( 續約 2)')
                if '100-1 全球不動產有價證券型（續約）' in page_text:
                    print('100-1 全球不動產有價證券型（續約） in ' + str(p+1) + '頁')
                    page_text = page_text.replace('100-1 全球不動產有價證券型（續約）','100-1 全球不動產有價證券型（續約）')
                    LPF_LIST.append('100-1 全球不動產有價證券型（續約）')
                if '102 年第二次委託經營(續約 2) (相對報酬)' in page_text:
                    print('102 年第二次委託經營(續約 2) (相對報酬) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('102 年第二次委託經營(續約 2) (相對報酬)','102 年第二次委託經營 ( 續約 )')
                    LPF_LIST.append('102 年第二次委託經營 ( 續約 )')
                if '103 年第一次委託經營第二期(續約)' in page_text:
                    print('103 年第一次委託經營第二期(續約) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('103 年第一次委託經營第二期(續約)','103 年第一次委託經營第二期(續約)')
                    LPF_LIST.append('103 年第一次委託經營第二期(續約)')
                if '103 年第二次委託經營(續約) (相對報酬)' in page_text:
                    print('103 年第二次委託經營(續約) (相對報酬) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('103 年第二次委託經營(續約) (相對報酬)','103 年第二次委託經營')
                    LPF_LIST.append('103 年第二次委託經營')
                if '104 年第一次委託經營  (續約)' in page_text:
                    print('104 年第一次委託經營  (續約) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('104 年第一次委託經營  (續約)','104 年第一次委託經營第一期( 續約 )')
                    LPF_LIST.append('104 年第一次委託經營第一期( 續約 )')
                if '106 年第一次委託經營' in page_text:
                    print('106 年第一次委託經營 in ' + str(p+1) + '頁')
                    page_text = page_text.replace('106 年第一次委託經營','106 年第一次委託經營')
                    LPF_LIST.append('106 年第一次委託經營')
                if '107 年第一次委託經營(相對報酬)' in page_text:
                    print('107 年第一次委託經營(相對報酬) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('107 年第一次委託經營(相對報酬)','107 年第一次委託經營')
                    LPF_LIST.append('107 年第一次委託經營')            
                if '107 年第二次委託經營' in page_text:
                    print('107 年第二次委託經營 in ' + str(p+1) + '頁')
                    page_text = page_text.replace('107 年第二次委託經營','107 年第二次委託經營')
                    LPF_LIST.append('107 年第二次委託經營')  
                if '109 年第一次委託經營' in page_text:
                    print('109 年第一次委託經營 in ' + str(p+1) + '頁')
                    page_text = page_text.replace('109 年第一次委託經營','109 年第一次委託經營')
                    LPF_LIST.append('109 年第一次委託經營')  
                 


            elif p == 11:
                print('到達12頁')
                if '110 年第一次委託經營' in page_text:
                    print('110 年第一次委託經營 in ' + str(p+1) + '頁')
                    page_text = page_text.replace('110 年第一次委託經營','110年第一次委託經營')
                    LPF_LIST.append('110年第一次委託經營')
                #舊制，將PDF檔案中的基金名稱改成excel內的，過渡期
                FUND_NAME = 'LRF'
                if '國內 98 年第二次委託經營(續約 4)' in page_text:
                    print('國內 98 年第二次委託經營(續約 4) in ' + str(p+1) + '頁')
                    page_text = page_text.replace('國內 98 年第二次委託經營(續約 4)','國內 98 年第二次委託經營( 續約 4)')
                    LRF_LIST.append('國內 98 年第二次委託經營( 續約 4)')
                if '97-1 全球增值債券型(續約 3)' in page_text:
                    print('97-1 全球增值債券型(續約 3)  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('97-1 全球增值債券型(續約 3)','97年續約3國外委託經營( 全球增值債券型 )')
                    LRF_LIST.append('97年續約3國外委託經營( 全球增值債券型 )')      
            elif p>11:
                FUND_NAME = 'LRF'      
                if '國內 101 年第一次委託經營(相對報酬)(續約 2)' in page_text:
                    print('國內 101 年第一次委託經營(相對報酬)(續約 2)  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('國內 101 年第一次委託經營(相對報酬)(續約 2)','國內 101 年第一次委託經營( 續約2 )')
                    LRF_LIST.append('國內 101 年第一次委託經營( 續約2 )') 
                if '國內 101 年第二次委託經營(續約 2)' in page_text:
                    print('國內 101 年第二次委託經營(續約 2)  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('國內 101 年第二次委託經營(續約 2)','國內 101 年第二次委託經營( 續約2 )')
                    LRF_LIST.append('國內 101 年第二次委託經營( 續約2 )')
                if '國內 104 年第一次委託經營(續約)' in page_text:
                    print('國內 104 年第一次委託經營(續約)  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('國內 104 年第一次委託經營(續約)','國內104 年第一次委託經營第一期( 續約 )')
                    LRF_LIST.append('國內104 年第一次委託經營第一期( 續約 )')
                if '國內 106 年第一次委託經營' in page_text:
                    print('國內 106 年第一次委託經營  in ' + str(p+1) + '頁')
                    page_text = page_text.replace('國內 106 年第一次委託經營','國內 106 年第一次委託經營第一期')
                    LRF_LIST.append('國內 106 年第一次委託經營第一期')
                
            
            for target_fund in list(LPF_LIST):
                #如果我的目標基金有在TEXT內
                if str(target_fund) != 'nan':
                    if target_fund in page_text:
                        if p<=11:
                            #小作弊，因為第五頁有三張表，第三張才是我要的
                            if p == 5:
                                table_num = 2
                            有新制勞退目標基金的頁數.append((p,table_num))
                            新制勞退目標基金的名稱.append(target_fund)
                            NEW_FUND_NAMES.append(FUND_NAME)
                            table_num+=1
            for target_fund in list(LRF_LIST):
                #如果我的目標基金有在TEXT內
                if str(target_fund) != 'nan':
                    if target_fund in page_text:
                        if p>=11:
                            #小作弊，因為第五頁有三張表，第三張才是我要的
                            if p == 5:
                                table_num = 2
                            有舊制勞退目標基金的頁數.append((p,table_num))
                            舊制勞退目標基金的名稱.append(target_fund)
                            OLD_FUND_NAMES.append(FUND_NAME)
                            table_num+=1

        LPF = pd.DataFrame({})
        PDF_file = pr.open(input_path+'/'+PDF_files[2])
        table_name_index = 0
        ps = PDF_file.pages
        for page,table_num in 有新制勞退目標基金的頁數:
            pg = ps[page]
            #取單頁全部的table
            page_tables = pg.extract_tables()
            try:
                table_勞退 = pd.DataFrame(page_tables[table_num])
                #處理表格內的資料
                table_勞退.iloc[0] = table_勞退.iloc[0].fillna('',axis=0)
                table_勞退.iloc[1] = table_勞退.iloc[1].fillna('',axis=0)
                Date_Start = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_勞退.iloc[0,4]).group(1)
                Date_End = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_勞退.iloc[0,4]).group(2)
                table_勞退.iloc[0,4] = re.sub(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)','',table_勞退.iloc[0,4])
                table_勞退_final = table_勞退.iloc[2:,:]
                table_勞退_final.columns = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target']
                table_勞退_final['Target'] = table_勞退_final['Target'].fillna(method = 'ffill')
                table_勞退_final[['Fund','Mandate Type','From','As of']]=['LPF',新制勞退目標基金的名稱[table_name_index],Date_Start,Date_End]
                table_勞退_final['From'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_Start)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_Start)[0][3:])).replace('.','/')
                table_勞退_final['As of'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_End)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_End)[0][3:])).replace('.','/')
                table_勞退_final = table_勞退_final[table_勞退_final['SITE']!='合計']
                table_name_index += 1
                LPF = LPF.append(table_勞退_final)
            except:
                print(str(page)+'有問題')
                table_name_index += 0
        LPF.reset_index(drop=True,inplace=True)

        LRF = pd.DataFrame({})
        PDF_file = pr.open(input_path+'/'+PDF_files[2])
        table_name_index = 0
        ps = PDF_file.pages
        for page,table_num in 有舊制勞退目標基金的頁數:
            pg = ps[page]
            #取單頁全部的table
            page_tables = pg.extract_tables()
            try:
                table_勞退 = pd.DataFrame(page_tables[table_num])
                #處理表格內的資料
                table_勞退.iloc[0] = table_勞退.iloc[0].fillna('',axis=0)
                table_勞退.iloc[1] = table_勞退.iloc[1].fillna('',axis=0)
                Date_Start = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_勞退.iloc[0,4]).group(1)
                Date_End = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_勞退.iloc[0,4]).group(2)
                table_勞退.iloc[0,4] = re.sub(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)','',table_勞退.iloc[0,4])
                table_勞退_final = table_勞退.iloc[2:,:]
                table_勞退_final.columns = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target']
                table_勞退_final['Target'] = table_勞退_final['Target'].fillna(method = 'ffill')
                table_勞退_final[['Fund','Mandate Type','From','As of']]=['LRF',舊制勞退目標基金的名稱[table_name_index],Date_Start,Date_End]
                table_勞退_final['From'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_Start)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_Start)[0][3:])).replace('.','/')
                table_勞退_final['As of'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_End)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_End)[0][3:])).replace('.','/')
                table_勞退_final = table_勞退_final[table_勞退_final['SITE']!='合計']
                table_name_index += 1
                LPF = LPF.append(table_勞退_final)
            except:
                print(str(page)+'有問題')
                table_name_index += 0
        LRF.reset_index(drop=True,inplace=True)

96 年第一次委託經營(續約 4) in 6頁
97 年第一次委託經營(續約 4) in 7頁
98 年第一次委託經營(續約 4) in 7頁
99 年第二次委託經營(續約 2) in 7頁
100 年第一次委託經營(續約 2)(相對報酬) in 7頁
101 年第一次委託經營(續約 2)(相對報酬) in 8頁
101 年第二次委託經營(續約 2) in 8頁
102 年第二次委託經營(續約 2) (相對報酬) in 8頁
103 年第一次委託經營第二期(續約) in 9頁
103 年第二次委託經營(續約) (相對報酬) in 9頁
104 年第一次委託經營  (續約) in 9頁
106 年第一次委託經營 in 10頁
107 年第一次委託經營(相對報酬) in 10頁
107 年第二次委託經營 in 11頁
109 年第一次委託經營 in 11頁
到達12頁
110 年第一次委託經營 in 12頁
國內 98 年第二次委託經營(續約 4) in 12頁
國內 101 年第一次委託經營(相對報酬)(續約 2)  in 13頁
國內 101 年第二次委託經營(續約 2)  in 13頁
國內 104 年第一次委託經營(續約)  in 13頁
國內 106 年第一次委託經營  in 14頁


In [66]:
NPIF_LIST = []
有國保目標基金的頁數 = []
國保目標基金的名稱   = []
NPIF_FUND_NAMES    = []
for PDF_file in PDF_files:
    if '國民年金保險' in PDF_file:
        國保_PDF = pr.open(input_path+'/'+PDF_file)
        #從0開始
        #是一個list
        ps              = 國保_PDF.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[p]
            page_text = pg.extract_text()
            #新制，將PDF檔案中的基金名稱改成excel內的
            FUND_NAME = 'NPIF'
            if '102 年度續約 1 委託帳戶' in page_text:
                print('102 年度續約 1 委託帳戶 in ' + str(p+1) + '頁')
                page_text = page_text.replace('102 年度續約 1 委託帳戶','國民年金保險基金102 年度委託帳戶續約1')
                NPIF_LIST.append('國民年金保險基金102 年度委託帳戶續約1')


            for target_fund in list(NPIF_LIST):
                #如果我的目標基金有在TEXT內
                if str(target_fund) != 'nan':
                    if target_fund in page_text:
                        有國保目標基金的頁數.append((p,table_num))
                        國保目標基金的名稱.append(target_fund)
                        NPIF_FUND_NAMES.append(FUND_NAME)
                        table_num+=1

    NPIF = pd.DataFrame({})
    PDF_file = pr.open(input_path+'/'+PDF_files[3])
    table_name_index = 0
    ps = PDF_file.pages
    for page, table_num in 有國保目標基金的頁數:
        pg = ps[page]
        page_tables = pg.extract_tables()
        try:
            table_國民年金 = pd.DataFrame(page_tables[table_num])
            #處理表格內的資料
            table_國民年金.iloc[0] = table_國民年金.iloc[0].fillna('',axis=0)
            table_國民年金.iloc[1] = table_國民年金.iloc[1].fillna('',axis=0)
            Date_Start = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_國民年金.iloc[0,4]).group(1)
            Date_End = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_國民年金.iloc[0,4]).group(2)
            table_國民年金.iloc[0,4] = re.sub(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)','',table_國民年金.iloc[0,4])
            table_國民年金_final = table_國民年金.iloc[2:,:]
            table_國民年金_final.columns = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target']
            table_國民年金_final['Target'] = table_國民年金_final['Target'].fillna(method = 'ffill')
            table_國民年金_final[['Fund','Mandate Type','From','As of']]=['NPIF',國保目標基金的名稱[table_name_index],Date_Start,Date_End]
            table_國民年金_final['From'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_Start)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_Start)[0][3:])).replace('.','/')
            table_國民年金_final['As of'] = (str(int(re.findall(r'\d+.\d+.\d+',Date_End)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_End)[0][3:])).replace('.','/')
            table_國民年金_final = table_國民年金_final[table_國民年金_final['SITE']!='合計']
            table_name_index += 1
        except:
            table_name_index += 0
        NPIF = NPIF.append(table_國民年金_final)

102 年度續約 1 委託帳戶 in 4頁


In [71]:
for PDF_file in PDF_files:
    if '基金運用情形' in PDF_file:
        退撫_PDF = pr.open(input_path+'/'+PDF_file)
        #從0開始
        #是一個list
        ps              = 退撫_PDF.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[3]
            page_text = pg.extract_text()
            print(page_text)

  107年委託經營（自 109年3月24日至 111年2月28日止） 
111年度 委託迄今 
受託機構名稱  委託金額  資產淨值  報酬率 報酬率 委託經 指標報酬率
(%)  (%)  營排名  (%) 
復華投信  2,000,000,000    3,413,997,927  -3.73   94.87  1  
統一投信  2,000,000,000    3,423,833,406  -3.52   94.70  2  
國泰投信  2,000,000,000    3,349,721,735  -3.45   94.30  3  
野村投信  2,000,000,000    3,332,511,643  -4.31   93.69  4   110.72 
群益投信  2,000,000,000    3,352,420,917  -3.14   92.50  5  
保德信投信  2,000,000,000   3,313,688,926  -3.15   88.45  6  
合計  12,000,000,000  20,186,174,554  -3.55  93.12    
註：本批次委託經營係追踨指數(相對報酬型)，以台灣證券交易所編製之發行量加權股價報酬指數之 
投資報酬率加50個基本點為年投資報酬目標。 
 
 
108年委託經營（自 109年2月7日至 111年2月28日止） 
111年度 委託迄今 
受託機構名稱  委託金額  資產淨值  報酬率 報酬率 委託經 目標報酬率
(%)  (%)  營排名  (%) 
安聯投信  5,000,000,000  7,098,161,390  -3.82   54.30  1  
野村投信  5,000,000,000  7,066,451,702  -5.41   51.37  2  
復華投信  4,000,000,000  5,959,203,800  -10.08   50.39  3   12.61 
國泰投信  5,000,000,000  6,960,304,047  -6.33   49.13  4  
合計  19,000,000,000  27,084,120,939  -6.31   51.22    
註：本批次委託經營係非追踨指數(絕對報酬型)，以台灣證券交易

In [72]:
PSPF_LIST = []
有退撫目標基金的頁數 = []
退撫目標基金的名稱   = []
PSPF_FUND_NAMES    = []
for PDF_file in PDF_files:
    if '基金運用情形' in PDF_file:
        退撫_PDF = pr.open(input_path+'/'+PDF_file)
        #從0開始
        #是一個list
        ps              = 退撫_PDF.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[p]
            page_text = pg.extract_text()
            #新制，將PDF檔案中的基金名稱改成excel內的
            FUND_NAME = 'PSPF'
            if '101年委託經營' in page_text:
                print('101年委託經營 in ' + str(p+1) + '頁')
                page_text = page_text.replace('101年委託經營','101年委託經營')
                PSPF_LIST.append('101年委託經營')
            if '102年委託經營' in page_text:
                print('102年委託經營 in ' + str(p+1) + '頁')
                page_text = page_text.replace('102年委託經營','102年委託經營')
                PSPF_LIST.append('102年委託經營')
            if '105年委託經營' in page_text:
                print('105年委託經營  in ' + str(p+1) + '頁')
                page_text = page_text.replace('105年委託經營','105年委託經營') 
                PSPF_LIST.append('105年委託經營')
            if '107年委託經營' in page_text:
                print('107年委託經營  in ' + str(p+1) + '頁')
                page_text = page_text.replace('107年委託經營','107年委託經營')    
                PSPF_LIST.append('107年委託經營')            
            if '108年委託經營' in page_text:
                print('108年委託經營 in ' + str(p+1) + '頁')
                page_text = page_text.replace('108年委託經營','108年委託經營')  
                PSPF_LIST.append('108年委託經營')       
            
            for target_fund in list(PSPF_LIST):
                #如果我的目標基金有在TEXT內
                if str(target_fund) != 'nan':
                    if target_fund in page_text:
                        有退撫目標基金的頁數.append((p,table_num))
                        退撫目標基金的名稱.append(target_fund)
                        PSPF_FUND_NAMES.append(FUND_NAME)
                        table_num+=1

    PSPF = pd.DataFrame({})
    PSPF_table_name_index = 0
    for PSPF_page,PSPF_table_num in 有退撫目標基金的頁數:
        pg = ps[PSPF_page]
        page_tables = pg.extract_tables()
        #print(len(page_tables))
        try:
            table_委託經營  = pd.DataFrame(page_tables[PSPF_table_num])
            multi          = [table_委託經營.iloc[0,:].dropna(axis=0),table_委託經營.iloc[1,:].fillna('')+table_委託經營.iloc[2,:].fillna('')]
            mindex         = pd.MultiIndex.from_product(multi,names=['Level1','Level2'])
            table_最終結果  = pd.DataFrame(columns = mindex, index = table_委託經營.iloc[3:,:].index)
            table_最終結果[table_委託經營.iloc[0,:].dropna(axis=0)[0]] = table_委託經營.iloc[3:,:].values

            #將資料補齊
            #找到基金的日期
            Funds_Date     = table_最終結果.columns.get_level_values(0)[0]
            table_最終結果  = table_最終結果[table_最終結果[Funds_Date,'受託機構名稱'] != '合計']
            table_最終結果[Funds_Date,'Fund'] = 'PSPF'
            table_最終結果[Funds_Date,'Mandate Type'] = Funds_Date.split('（')[0]
            #將報酬率補起來，有的是目標，有的是指標
            try:
                table_最終結果[Funds_Date,'目標報酬率\n(%)'] = table_最終結果[Funds_Date,'目標報酬率\n(%)'].fillna(method = 'ffill')
            except:
                table_最終結果[Funds_Date,'指標報酬率\n(%)'] = table_最終結果[Funds_Date,'指標報酬率\n(%)'].fillna(method = 'ffill')
            table_最終結果[Funds_Date,'受託機構名稱'] = table_最終結果[Funds_Date,'受託機構名稱'].str.replace('投信','')
            #補開始及結束的日期
            table_最終結果[Funds_Date,'From']  = (str(int(re.findall(r'\d+年\d+月\d+日',Funds_Date)[0][:3])+1911)+str(re.findall(r'\d+年\d+月\d+日',Funds_Date)[0][3:])).replace('日','')
            table_最終結果[Funds_Date,'As of'] = (str(int(re.findall(r'\d+年\d+月\d+日',Funds_Date)[1][:3])+1911)+str(re.findall(r'\d+年\d+月\d+日',Funds_Date)[1][3:])).replace('日','')
            for target in ['年','月']:
                table_最終結果[Funds_Date,'From']  = table_最終結果[Funds_Date,'From'].str.replace(target,'/')
                table_最終結果[Funds_Date,'As of'] = table_最終結果[Funds_Date,'As of'].str.replace(target,'/')
            PSPF = PSPF.append(pd.DataFrame(table_最終結果.values))
        except:
            pass
    PSPF.columns = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target','Fund','Mandate Type','From','As of']

101年委託經營 in 3頁
102年委託經營 in 3頁
105年委託經營  in 3頁
107年委託經營  in 4頁
108年委託經營 in 4頁


In [73]:
PSPF

,SITE,Mandate Size,AUM,YTD %,Since Inception %,Ranking,Target,Fund,Mandate Type,From,As of
0,安聯,"6,000,000,000","20,952,509,328",-4.84,345.55,–,267.04,PSPF,101年委託經營,2012/6/5,2022/2/28
0,統一,"5,000,000,000","15,627,562,115",-2.57,211.70,1,155.23,PSPF,102年委託經營,2014/9/26,2022/2/28
1,安聯,"5,000,000,000","14,492,387,707",-4.84,198.65,2,155.23,PSPF,102年委託經營,2014/9/26,2022/2/28
0,保德信,"3,000,000,000","6,038,550,418",-2.84,109.23,1,98.27,PSPF,105年委託經營,2018/10/16,2022/2/28
1,復華,"3,000,000,000","5,922,791,551",-4.19,102.94,2,98.27,PSPF,105年委託經營,2018/10/16,2022/2/28
2,國泰,"3,000,000,000","5,702,690,805",-3.77,98.43,3,98.27,PSPF,105年委託經營,2018/10/16,2022/2/28
3,匯豐中華,"3,000,000,000","5,619,028,476",-2.19,95.51,4,98.27,PSPF,105年委託經營,2018/10/16,2022/2/28
0,復華,"2,000,000,000","3,413,997,927",-3.73,94.87,1,110.72,PSPF,107年委託經營,2020/3/24,2022/2/28
1,統一,"2,000,000,000","3,423,833,406",-3.52,94.70,2,110.72,PSPF,107年委託經營,2020/3/24,2022/2/28
2,國泰,"2,000,000,000","3,349,721,735",-3.45,94.30,3,110.72,PSPF,107年委託經營,2020/3/24,2022/2/28


In [76]:
for PDF_file in PDF_files:
    if '勞工保險' in PDF_file:
        勞保_PDF = pr.open(input_path+'/'+PDF_file)
        #從0開始
        #是一個list
        ps              = 勞保_PDF.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[3]
            page_text = pg.extract_text()
print(page_text[-600:])

均殖利率加計
200個基本點為基準。 
 
                  勞工保險基金 106 年度第 1 次       
單位：新臺幣元 
委任迄今 
111年截至2月
(107.10.8~111.2.28) 
投信名稱  委託金額  目前淨資產  累積投資報酬率
投資  目標報酬率
(％)  排名 
報酬率(％)  (％) 
台新  1,000,000,000  1,609,895,444  -4.59  60.99  2 
安聯  1,000,000,000  1,776,753,338  -3.60  77.68  1 
保德信  1,000,000,000  1,376,867,836  -3.42  37.69  6 
國泰  900,000,000  1,258,137,844  0.44  41.33  5  20.82 
群益  1,000,000,000  1,436,964,017  -4.98  43.70  4 
滙豐中華  1,000,000,000  1,458,414,897  -0.43  45.84  3 
合計  5,900,000,000  8,917,033,376  -2.92  51.21  - 
註：本批次為絕對報酬型，契約目標報酬率係以臺灣證交所公告之最近5年年底股票集中市場平均殖利率加計
200個基本點為基準。 
4 
 


In [82]:
LIF_LIST = []
有勞保目標基金的頁數 = []
勞保目標基金的名稱   = []
LIF_FUND_NAMES    = []
for PDF_file in PDF_files:
    if '勞工保險' in PDF_file:
        勞保_PDF = pr.open(input_path+'/'+PDF_file)
        #從0開始
        #是一個list
        ps              = 勞保_PDF.pages
        #用range配len跑for迴圈
        for p in range(len(ps)):
            table_num = 0
            pg = ps[p]
            page_text = pg.extract_text()
            #新制，將PDF檔案中的基金名稱改成excel內的
            FUND_NAME = 'LIF'
            if '勞工保險基金 103 年度續約 3' in page_text:
                print('勞工保險基金 103 年度續約 3 in ' + str(p+1) + '頁')
                page_text = page_text.replace('勞工保險基金 103 年度續約 3','勞工保險基金 103年度續約3')
                LIF_LIST.append('勞工保險基金 103年度續約3')
            if '勞工保險基金 104 年度續約 3' in page_text:
                print('勞工保險基金 104 年度續約 3 in ' + str(p+1) + '頁')
                page_text = page_text.replace('勞工保險基金 104 年度續約 3','勞工保險基金 104年度續約3')
                LIF_LIST.append('勞工保險基金 104年度續約3')
            if '勞工保險基金 106 年度第 1 次' in page_text:
                print('勞工保險基金 106 年度第 1 次  in ' + str(p+1) + '頁')
                page_text = page_text.replace('勞工保險基金 106 年度第 1 次','勞工保險基金 106 年度年第一次委託經營') 
                LIF_LIST.append('勞工保險基金 106 年度年第一次委託經營')       
            
                
            page_text = page_text.replace('\n','')
            for target_fund in list(LIF_LIST):
                #如果我的目標基金有在TEXT內
                if str(target_fund) != 'nan':
                    if target_fund in page_text:
                        有勞保目標基金的頁數.append((p,table_num))
                        勞保目標基金的名稱.append(target_fund)
                        LIF_FUND_NAMES.append(FUND_NAME)
                        table_num+=1

    LIF = pd.DataFrame({})
    PDF_file = pr.open(input_path+'/'+PDF_files[1])
    table_num_index=0
    for page_table, table_num in 有勞保目標基金的頁數:
        ps = PDF_file.pages
        pg = ps[page_table]
        table = pg.extract_tables()[table_num]
        table_勞保 = pd.DataFrame(table)
        #處理表格內的資料
        table_勞保.iloc[0] = table_勞保.iloc[0].fillna('',axis=0)
        table_勞保.iloc[1] = table_勞保.iloc[1].fillna('',axis=0)
        Date_Start = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_勞保.iloc[0,4]).group(1)
        Date_End   = re.search(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)',table_勞保.iloc[0,4]).group(2)
        table_勞保.iloc[0,4] = re.sub(r'(\d+.\d+.\d+)~(\d+.\d+.\d+)','',table_勞保.iloc[0,4])
        table_勞保_final           = table_勞保.iloc[2:,:]
        table_勞保_final.columns   = ['SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target']
        table_勞保_final['Target'] = table_勞保_final['Target'].fillna(method = 'ffill')
        table_勞保_final[['Fund','Mandate Type','From','As of']]=['LIF',勞保目標基金的名稱[table_num_index],Date_Start,Date_End]
        table_勞保_final['From']   = (str(int(re.findall(r'\d+.\d+.\d+',Date_Start)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_Start)[0][3:])).replace('.','/')
        table_勞保_final['As of']  = (str(int(re.findall(r'\d+.\d+.\d+',Date_End)[0][:3])+1911)+str(re.findall(r'\d+.\d+.\d+',Date_End)[0][3:])).replace('.','/')
        table_勞保_final = table_勞保_final[table_勞保_final['SITE']!='合計']
        table_num_index+=1
        LIF = LIF.append(table_勞保_final)

勞工保險基金 103 年度續約 3 in 4頁
勞工保險基金 104 年度續約 3 in 4頁
勞工保險基金 106 年度第 1 次  in 4頁


In [83]:
LIF

,SITE,Mandate Size,AUM,YTD %,Since Inception %,Ranking,Target,Fund,Mandate Type,From,As of
2,國泰,"4,280,395,006","5,854,820,756",-3.09,36.78,－,8.28,LIF,勞工保險基金 103年度續約3,2020/9/30,2022/2/28
2,野村,"8,518,656,861","9,295,090,558",-5.95,9.11,1,3.79,LIF,勞工保險基金 104年度續約3,2021/7/2,2022/2/28
3,永豐,"2,788,127,929","2,771,713,567",-6.72,-0.59,2,3.79,LIF,勞工保險基金 104年度續約3,2021/7/2,2022/2/28
2,台新,"1,000,000,000","1,609,895,444",-4.59,60.99,2,20.82,LIF,勞工保險基金 106 年度年第一次委託經營,2018/10/8,2022/2/28
3,安聯,"1,000,000,000","1,776,753,338",-3.60,77.68,1,20.82,LIF,勞工保險基金 106 年度年第一次委託經營,2018/10/8,2022/2/28
4,保德信,"1,000,000,000","1,376,867,836",-3.42,37.69,6,20.82,LIF,勞工保險基金 106 年度年第一次委託經營,2018/10/8,2022/2/28
5,國泰,"900,000,000","1,258,137,844",0.44,41.33,5,20.82,LIF,勞工保險基金 106 年度年第一次委託經營,2018/10/8,2022/2/28
6,群益,"1,000,000,000","1,436,964,017",-4.98,43.70,4,20.82,LIF,勞工保險基金 106 年度年第一次委託經營,2018/10/8,2022/2/28
7,滙豐中華,"1,000,000,000","1,458,414,897",-0.43,45.84,3,20.82,LIF,勞工保險基金 106 年度年第一次委託經營,2018/10/8,2022/2/28


In [96]:
GFI['新舊'] = 'old'
v=Final_df = pd.concat([LPF,LRF,LIF,NPIF,PSPF],axis=0)
Final_df.reset_index(drop=True,inplace=True)
Final_df['SITE'] = Final_df['SITE'].str.replace('\n','')
Final_df['SITE'] = Final_df['SITE'].str.replace('滙豐中華','匯豐中華')
Final_df = Final_df[['Fund','Mandate Type','SITE','Mandate Size','AUM','YTD %','Since Inception %','Ranking','Target','From','As of']]
Final_df.to_csv(r'D:\My Documents\andyhs\桌面\Andy\政府基金運用資訊-202112\output\page2_final_check.csv',encoding='UTF-8-SIG',index=False,header=True)

In [97]:
新舊合併 = pd.concat([Final_df,GFI],axis=0)
新舊合併 = 新舊合併[pd.isna(新舊合併['Fund'])==False]
for col in ['Fund','Mandate Type','SITE']:
    新舊合併[col] = 新舊合併[col].str.strip()
新舊合併_去重複 = 新舊合併.drop_duplicates(subset = ['Fund','Mandate Type','SITE'],keep = 'first')
新舊合併_去重複.reset_index(drop=True,inplace=True)
新舊合併_去重複.to_csv(r'D:\My Documents\andyhs\桌面\Andy\政府基金運用資訊-202112\output\page2_下半部有old標記為可能已刪除的.csv',encoding='UTF-8-SIG',index=False,header=True)

In [98]:
a = pd.read_csv(r'D:\My Documents\andyhs\桌面\Andy\政府基金運用資訊-202112\output\page2_下半部有old標記為可能已刪除的.csv')

In [102]:
新舊合併_去重複_new = 新舊合併_去重複[新舊合併_去重複['新舊'] != 'old']

In [107]:
GFI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Fund               94 non-null     object        
 1   Mandate Type       94 non-null     object        
 2   Asset Class        94 non-null     object        
 3   SITE               94 non-null     object        
 4   Mandate Size       95 non-null     int64         
 5   AUM                95 non-null     int64         
 6   YTD %              94 non-null     float64       
 7   Since Inception %  94 non-null     float64       
 8   Ranking            94 non-null     float64       
 9   Target             94 non-null     float64       
 10  From               94 non-null     datetime64[ns]
 11  As of              94 non-null     datetime64[ns]
 12  benchmark          1 non-null      object        
 13  新舊                 95 non-null     object        
dtypes: datetime6

In [114]:
新舊合併_去重複_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92 entries, 0 to 91
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Fund               92 non-null     object
 1   Mandate Type       92 non-null     object
 2   SITE               92 non-null     object
 3   Mandate Size       92 non-null     object
 4   AUM                92 non-null     object
 5   YTD %              92 non-null     object
 6   Since Inception %  92 non-null     object
 7   Ranking            92 non-null     object
 8   Target             92 non-null     object
 9   From               92 non-null     object
 10  As of              92 non-null     object
 11  Asset Class        1 non-null      object
 12  benchmark          0 non-null      object
 13  新舊                 0 non-null      object
dtypes: object(14)
memory usage: 12.8+ KB


In [115]:
#先skip
a = 0
match  = []
for x in range(len(新舊合併_去重複_new)):
    for y in range(len(GFI)):
        if 新舊合併_去重複_new.loc[x,'Fund'] == GFI.loc[y,'Fund'] and 新舊合併_去重複_new.loc[x,'Mandate Type'] == GFI.loc[y,'Mandate Type'] and 新舊合併_去重複_new.loc[x,'SITE'] == GFI.loc[y,'SITE']:
            #排名不更新
            a+=1
            match.append((x,y))
            if 新舊合併_去重複_new.loc[x,'Ranking'] == '－' or 新舊合併_去重複_new.loc[x,'Ranking'] == '–' or 新舊合併_去重複_new.loc[x,'Ranking'] == '-' or Final_df.loc[x,'Ranking'] == '－':
                print('ranking no change')
                要更新的資料 = list(GFI.loc[y,['Asset Class','Ranking']].values)
                新舊合併_去重複_new.loc[x,['Asset Class','Ranking']] = 要更新的資料
            else:
                print('ranking change')
                要更新的資料 = list(GFI.loc[y,['Asset Class']].values)
                新舊合併_去重複_new.loc[x,['Asset Class']] = 要更新的資料
        else:
            pass

ranking no change
ranking no change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking no change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking no change
ranking no change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking no change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking change
ranking

In [116]:
新舊合併_去重複_new.to_csv(r'D:\My Documents\andyhs\桌面\Andy\政府基金運用資訊-202112\output\page2.csv',encoding='UTF-8-SIG',index=False,header=True)